In [ ]:
import functools

import xarray as xr

import lifetimes

In [ ]:
local_path = "/home/fabian/Documents/MAELSTROM/data/pca/"
outfile = f"{local_path}/daily_averages_jan_dec_2020.nc"
months = [f"{i:02d}" for i in range(1, 13)]
paths = [f"{local_path}/ml_2020{month}01_00.nc" for month in months]
paths

In [ ]:
def preprocess(path: str) -> xr.Dataset:
    d = xr.open_dataset(path, engine="netcdf4")
    # Drop all variables except temperature.
    d = d.drop(["q", "u", "v"])
    # Selcect one exemplary model level and drop this dimension.
    d = d.isel(level=0).drop("level")
    # Slice data after 48 hours (model runs are for 48h)
    d = lifetimes.utils.slice_dataset(
        d,
        dimension="time",
        slice_until=48,
    )
    d = lifetimes.utils.calculate_daily_averages(d, time_coordinate="time")
    return d

In [ ]:
ds = preprocess(paths[0])
for path in paths[1:]:
    dataset = preprocess(path)
    ds = xr.concat([ds, dataset], dim="time")

In [ ]:
ds["time"].values

In [ ]:
ds.to_netcdf(outfile)

In [ ]:
ds = xr.open_dataset(outfile, engine="netcdf4")

In [ ]:
ds